In [17]:
import os
from bs4 import BeautifulSoup
import time
import datetime
import pandas as pd
import requests
import re


# 크롤링
def do_crawling():
    # 섹션 별 아이디
    Section_ID = {"Political":"100", "Economic":"101", "Society":"102", "Culture":"103", "International":"104", "Science":"105"}

    # 크롤링 시간 측정
    start = time.time()
    now = datetime.datetime.now()
    date = "".join(str(now.date()).split('-'))
    
    # 파일 저장 경로 설정
    directory = "C:\\Users\\cjy89\\NLP\\Project_news_crawling\\Naver\\" + str(date)
    if not os.path.isdir(directory):
        os.makedirs(directory)
    os.chdir(directory)
    
    # csv 파일 이름 설정
    title = str(now.hour) + ".csv"

    # 서버에서 뉴스 데이터 가져오기
    page = 0
    News_DataFrame = pd.DataFrame()

    flag = True
    while(flag):
        page += 1
        
        # 제목형으로 정치 섹션 데이터 긁어오기
        url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&listType=title&sid1=" + Section_ID["Political"] + "&date=" + date + "&page=" + str(page)
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")

        # news_box 한 요소당 5개의 뉴스가 포함됨
        news_box = soup.find_all(class_ = "type02")
        for box in range(len(news_box)):
            news = [i.get_text() for i in news_box[box].find_all(class_="nclicks(fls.list)")]
            urls = [i['href'] for i in news_box[box].find_all(class_="nclicks(fls.list)")]
            press = [i.get_text() for i in news_box[box].find_all(class_="writing")]
            upload = [ str(now.date()) + ' ' + str(now.hour) + ':' + str(now.minute - int(i.get_text()[:-2])) 
                      for i in news_box[box].find_all(class_="date is_new") if i.get_text() not in '시간']

            # 1시간이 넘어간 뉴스는 긁어오지 않음 (발견되면 크롤링 종료)
            if len(upload) != 5:
                flag = False
                news = news[:len(upload)]
                urls = urls[:len(upload)]
                press = press[:len(upload)]

            temp_df = pd.DataFrame({"News":news, "Url":urls, "Press":press, "Time":upload})
            News_DataFrame = pd.concat([News_DataFrame, temp_df], axis=0, ignore_index=True)
            News_DataFrame.dropna(axis=0, inplace=True)

    # 데이터 프레임 csv 파일로 저장
    end = time.time()
    News_DataFrame.to_csv(title, sep=",", encoding='utf-8-sig')
    print("Crawling " + date + " " + title + " Time :", end-start)

In [18]:
do_crawling()

Crawling 20210206 14.csv Time : 0.1687757968902588


In [33]:
# 크롤링 자동화
import schedule
import datetime
import pause

now = datetime.datetime.now()
wait_time = datetime.datetime(now.year, now.month, now.day, now.hour, 59, 55)
pause.until(wait_time)

schedule.every().hour.do(do_crawling)
while True:    
    schedule.run_pending()
    time.sleep(1)    

Crawling 20210206 16.csv Time : 0.478715181350708
Crawling 20210206 16.csv Time : 0.3484020233154297
Crawling 20210206 16.csv Time : 0.4167048931121826
Crawling 20210206 16.csv Time : 0.5725107192993164
Crawling 20210206 16.csv Time : 0.501666784286499
Crawling 20210206 16.csv Time : 0.6722018718719482
Crawling 20210206 16.csv Time : 0.5225124359130859
Crawling 20210206 16.csv Time : 0.5414810180664062
Crawling 20210206 16.csv Time : 0.5724697113037109
Crawling 20210206 16.csv Time : 0.6637814044952393
Crawling 20210206 17.csv Time : 0.21230196952819824
Crawling 20210206 17.csv Time : 0.18146681785583496
Crawling 20210206 17.csv Time : 0.3755831718444824
Crawling 20210206 17.csv Time : 0.24933409690856934
Crawling 20210206 17.csv Time : 0.17054533958435059
Crawling 20210206 17.csv Time : 0.48415493965148926
Crawling 20210206 17.csv Time : 0.1976611614227295
Crawling 20210206 17.csv Time : 0.1889331340789795
Crawling 20210206 17.csv Time : 0.39434289932250977


KeyboardInterrupt: 